<a href="https://colab.research.google.com/github/marcelo-cueto/Blockchain_python/blob/main/Blockchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install flask-ngrok

In [ ]:
import datetime
import hashlib
import json
from flask  import Flask, jsonify
from flask_ngrok import run_with_ngrok

In [ ]:
class Blockchain:

  def __init__(self) :
      """Constructor del Blockchain"""
      self.chain = []
      self.create_block(proof=1, previous_hash= '0')

  def create_block(self, proof, previous_hash):
    """Cracion de un nuevo Bloque:

    Arguments:
      -proof:Nounce del Bloque actual
      -previous_hash: Hash del bloque previo
      
    Returns:
      -Block: Devolucion Nuevo bloque creado.
      """
    block={
            'index'         : len(self.chain)+1,
            'Timestamp'     : str(datetime.datetime.now()),
            'proof'         : proof,
            'previous_hash' : previous_hash
          }
    self.chain.append(block)
    return block
  
  def get_previous_block(self):
    """Obtencion del bloque anterior
    Return:
      -Devolucion del ultimo bloque de la blockchain.
    """
    return self.chain[-1]

  def proof_of_work(self, previous_proof):
    """Protocolo de concenco (PoW)

    Arguments:
      -previous_proof: Hash del bloque previo

    Return:
      -new_proof: Devolucion del nuevo hash obtenido del PoW
    """
    new_proof=1

    chech_proof=False

    while chech_proof == False:
      hash_operation = hashlib.sha256(str(new_proof**2-previous_proof**2).encode()).hexdigest()

      if hash_operation[:4] == '0000':

        chech_proof == True

      else:
         
        new_proof += 1

    return new_proof
  
  def hash(self, block):
    """Calculo del Hash de un bloque
    
    Arguments:
      -block : Identifica a un bloque de Blockchain

    Returns:
      -hash_block: Devuelve el hash del bloque
    """

    encoded_block = json.dumps(block, sort_keys=True).encode()
    hash_block = hashlib.sha256(encoded_block).hexdigest()

    return hash_block
  
  def is_chain_valid(self, chain):
    """Verifica si la Blockchain es valida
    
    Arguments:
      -chain : cadena de bloques que contiene toda la informacion de las transacciones

    Returns:
      -True/False: Devuelve un booleano en funcion de la validez de la blockchain
                  (True: valido , False:Invalido)
    """
    previous_block = chain[0]
    block_index=1

    while block_index < len(chain):

      block = chain[block_index]
      
      if block['previous_hash'] != self.hash(previous_block):

        return False
      
      previous_proof = previous_block['proof']
      proof= block['proof']
      hash_operation = hashlib.sha256(str(proof**2-previous_proof**2).encode()).hexdigest()

      if hash_operation[:4] != '0000':
        
        return False

      previous_block = block
      block_index += 1
    
    return True




In [ ]:
#Crear una app web
#Ejecucion de la app con Flask
app = Flask(__name__)
run_with_ngrok(app)

#Si se obtiene un error 500, actualizar flask y ejecutar la siguiente linea
app.config['JSONIFY_PRETTYPRINT_REGULAR'] = False

#Creacion de la Blockchain
blockchain = Blockchain()

@app.route('/mine_block', methods =['GET'])
def mine_block():
  """Minado de un bloque"""

  previous_block = blockchain.get_previous_block()
  previous_proof = previous_block['proof']
  proof = blockchain.proof_of_work(previous_proof)
  previous_hash = blockchain.hash(previous_block)
  block = blockchain.create_block(proof, previous_hash)
  response = {'message'       :'Good work, a block was mined!',
              'index'         : block['index'],
              'Timestamp'     : block['Timestamp'],
              'proof'         : block['proof'],
              'previous_hash' : block['previous_hash'],                  
             }
  return jsonify(response), 200


@app.route('/get_chain', methods = ['GET'])
def get_chain():
  """ Obtencion de Blockchain """
  response = {'(chain'    : blockchain.chain,
              'length'   : len(blockchain.chain)       
             }
  return jsonify(response), 200

@app.route('/is_valid', methods = ['GET'])
def is_valid():
  """ Comprovasion si la Blockchain es valida """

  is_valid = blockchain.is_chain_valid(blockchain.chain)
  if is_valid:
    response = {'message' : 'The Blockchain es valida'}
  else:
    response = {'message' : 'The Blockchain NO es valida'}
  return jsonify(response), 200




In [ ]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/lib/python3.7/http/client.py",